
# Messages from the Mayor Analysis

Data is harvest/constructed in this [notebook](https://github.com/BrockDSL/ARCH_Data_Explore/blob/main/Junkin_Processing.ipynb)

Originally found here: [https://www.pelham.ca/en/living-here/covid-19-messages-from-the-mayor.aspx](https://www.pelham.ca/en/living-here/covid-19-messages-from-the-mayor.aspx)

Now deleted. April 21, 2021 last [snapshot](https://web.archive.org/web/20210422114625/https://www.pelham.ca/en/living-here/covid-19-messages-from-the-mayor.aspx)

This notebook only works as a Jupyter notebook. It also won't work in Jupyter-lab and needs the classic notebook view. This is because of the Javascript requirements (the IPython.WidgetManager is not available in Jupyter-lab or Google Colab).  

In [ ]:
!pip install ipywidgets pandas requests textblob nltk flair

In [1]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd
import requests
import os

from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
#classifier_rnn_fast = TextClassifier.load('sentiment-fast')

# Import flair Sentence to process input text
from flair.data import Sentence

pd.set_option('display.max_colwidth', 999)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


2022-12-04 22:45:36,692 loading file /home/jovyan/.flair/models/sentiment-en-mix-distillbert_4.pt


In [2]:
#Sentiment score from the classifiers is always a positive number indicating probability of the pred={POSITIVE,NEGATIVE}
#Treat the score negative if pred=NEGATIVE and treat the score as a % of positivity/negativity
def rescale_sentiment(sentence):
  score = sentence.labels[0].score
  pred = sentence.labels[0].value
  
  if pred == 'POSITIVE': return score
  else: return score * -1

def flair_sentiment(texts, classifier):
    sentences = [Sentence(text) for text in texts]
    classifier.predict(sentences, mini_batch_size=32, verbose=True)
    return [rescale_sentiment(sent) for sent in sentences]

In [3]:
if os.path.exists('junkin.csv.gz'):
    junkin_data = pd.read_csv('junkin.csv.gz')
else:
    junkin_data = pd.read_csv("https://raw.githubusercontent.com/BrockDSL/ARCH_Data_Explore/main/Junkin_Posts.csv")
    junkin_data['vader_comp'] = junkin_data.apply(lambda row: sid.polarity_scores(row.Entry)['compound'], axis=1)
    junkin_data['text_blob'] = junkin_data.apply(lambda row: TextBlob(row.Entry).sentiment.polarity, axis=1)
    junkin_data['bert'] = flair_sentiment(junkin_data.Entry, classifier)
    #junkin_data['rnn'] = flair_sentiment(junkin_data.Entry, classifier_rnn_fast)
    
junkin_data['Date'] = pd.to_datetime(junkin_data['Date'])

In [4]:
plt.ioff() #turn off automatic display of plots
#algorithms = ['vader_comp','text_blob','bert','rnn']
algorithms = ['vader_comp','text_blob','bert']

def plot_sentiment(algorithm):
    
    fig, ax = plt.subplots(figsize=(12,8))

    ax.bar(junkin_data["Date"],junkin_data[algorithm])
    ax.set_title("Sentiment of 'Messages from the Mayor'")
    ax.set_ylabel("Sentiment")
    ax.set_xlabel("Date")

interact(plot_sentiment, algorithm=algorithms)

interactive(children=(Dropdown(description='algorithm', options=('vader_comp', 'text_blob', 'bert'), value='va…

<function __main__.plot_sentiment(algorithm)>

In [5]:
import ipywidgets as widgets
from ipywidgets import Layout, VBox, HBox
from IPython.display import clear_output
from IPython.display import Javascript

def display_hbar_sentiment(data, title, output):
    with output:
        output.clear_output()
        
        fig, ax = plt.subplots() 

        if type(data) == str:
            text = data
            vc = sid.polarity_scores(text)['compound']
            tb = TextBlob(text).sentiment.polarity
            bert = flair_sentiment([text], classifier)
            #rnn = flair_sentiment([text], classifier_rnn_fast)
        else:
            row_data = data
            vc = row_data['vader_comp']
            tb = row_data['text_blob']
            bert = row_data['bert']
            #rnn = row_data['rnn']

        #ax.barh(range(4),[vc,tb,bert,rnn], tick_label=['vader','textblob','bert','Rnn'])
        ax.barh(range(3),[vc,tb,bert], tick_label=['vader','textblob','bert'])
        ax.set_title(title)
        ax.set_xlabel("Sentiment")
        display(fig)
    
    
def sentiment_explore(x):
    row_data = junkin_data.iloc[x]
    display_hbar_sentiment(row_data, 'Entire message sentiment', o2)
    display_hbar_sentiment(row_data, 'Selected text sentiment', o3)
    text.value = row_data.Entry
    
text = widgets.Textarea(
    value='',
    description='Entry:',
    disabled=False,
    layout=Layout(width='95%', height='200px')
)
sel_text = widgets.Text(
    value='',
    description='selected text:',
    disabled=False,
    visible=False,
    layout=Layout(width='95%')
)

def on_value_change(change):
    #o3.clear_output()
    if change['old'] != change['new']: 
        display_hbar_sentiment(change['new'], 'Selected text sentiment', o3)

sel_text.observe(on_value_change, names='value')
jscript = f'''

function put2widget(arg) {{
    var manager = window.IPython.WidgetManager._managers[0];
    var ta = manager.get_model('{sel_text.model_id}');
    ta.then(function(model) {{
        model.set('value', arg)
        model.save()
    }});
}}

function getSelection(event) {{
  
  const selection = event.target.value.substring(event.target.selectionStart, event.target.selectionEnd);
  document.querySelectorAll('input[type="text"]')[0].value=selection;
  put2widget(selection)

}}

const area = document.querySelectorAll('textarea.widget-input')[0]
area.addEventListener("select", getSelection); 

'''


o2 = widgets.Output(layout={'border': '1px solid black'})
o3 = widgets.Output(layout={'border': '1px solid black'})

entry_slider = widgets.IntSlider(description="Entry #: ", min=0, max=len(junkin_data)-1, step=1, value=0)

#This sets up a callback when changes are made to the slider
widgets.interactive_output(sentiment_explore, {'x':entry_slider})


display(text, entry_slider, sel_text, HBox([o2,o3]))

#Must call this after all elements exist in the DOM
Javascript(jscript)


Textarea(value='On Friday the Provincial government introduced new measures to try to halt the runaway third w…

IntSlider(value=0, description='Entry #: ', max=53)

Text(value='', description='selected text:', layout=Layout(width='95%'))

<IPython.core.display.Javascript object>

In [6]:
plt.ioff() #turn off automatic display of plots
custom_text = widgets.Textarea(
    value='Enter text to analyze for sentiment here.',
    description='Entry:',
    disabled=False,
    layout=Layout(width='95%', height='200px')
)
button = widgets.Button(
    description='Analyze',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button

def custom_barh(cb):
    text = custom_text.value
    display_hbar_sentiment(text, 'Sentiment', custom_out)
    
custom_out = widgets.Output(layout={'border': '1px solid black'})
button.on_click(custom_barh)
display(custom_text, button, custom_out)

    

Textarea(value='Enter text to analyze for sentiment here.', description='Entry:', layout=Layout(height='200px'…

Button(description='Analyze', icon='check', style=ButtonStyle(), tooltip='Click me')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…